#### Develop an agent function to process a list of WhatsApp messages and extract events.

1. Load the WhatsApp messages from a file.
2. Process the messages to extract events.
3. Visualize the events in a table.

In [15]:
# Libraries & functions to pretty print text and JSON responses, just needed for the notebook
import textwrap # Text wrapping
from pprint import pprint # Pretty printing json

def pretty_print(response):
    if isinstance(response, (str, dict)):
        print(textwrap.fill(response, width=80))
    else:
        response_dict = response.model_dump()
        pprint(response_dict, width=80)

In [16]:
# Open the file

with open('../dummy-data/test-file.txt', 'r', encoding='utf-8') as file:
    sample_message_history = file.read()


In [17]:
pretty_print(sample_message_history)

[01/10/2023, 10:15 AM] Isabel: Hey everyone! We should plan to watch the match
on Tuesday! Who's in? ⚽️ [01/10/2023, 10:16 AM] Ana: I'm in! Can't wait to see
the game! 🙌 [01/10/2023, 10:17 AM] Carmen: I have plans this weekend, so I can't
make it. 😞 [01/10/2023, 10:18 AM] Julia: Count me in! Where are we watching it?
[01/10/2023, 10:19 AM] Laura: I’ll join too! Let’s make it a fun day! 🎉
[01/10/2023, 10:20 AM] Sofia: I might be able to come, but I’ll confirm later.
[01/10/2023, 10:21 AM] Isabel: Great! Let’s meet at my place then.  [01/10/2023,
10:22 AM] Isabel: Also, there’s a friendly match next Friday. Anyone interested?
[01/10/2023, 10:23 AM] Ana: I can’t make it on Friday. I have a meeting. 😩
[01/10/2023, 10:24 AM] Carmen: I’m free! I’d love to join!  [01/10/2023, 10:25
AM] Julia: I’ll be there too!  [01/10/2023, 10:26 AM] Laura: I can’t do
Wednesday, unfortunately.  [01/10/2023, 10:27 AM] Sofia: I’ll join for the
friendly match!  [01/10/2023, 10:28 AM] Isabel: Awesome! So far we 

In [18]:
from datetime import datetime
import re

def format_message_history(sample_message_history):
    # Define a regex pattern to find timestamps in the format [DD/MM/YY, HH:MM:SS]
    pattern = r'\[(\d{2}/\d{2}/\d{2}), (\d{2}:\d{2}:\d{2})\]'
    
    # Function to replace each timestamp with the desired format
    def replace_timestamp(match):
        date_str = match.group(1)  # Get the date part
        time_str = match.group(2)  # Get the time part
        # Combine date and time for parsing
        full_datetime_str = f"{date_str} {time_str}"
        # Parse the datetime
        message_datetime = datetime.strptime(full_datetime_str, "%d/%m/%y %H:%M:%S")
        # Get the day of the week
        day_of_week = message_datetime.strftime("%A")
        # Return the formatted timestamp with the day of the week
        return f"[{date_str}, {day_of_week} {time_str}]"

    # Replace all timestamps in the message history
    formatted_message_history = re.sub(pattern, replace_timestamp, sample_message_history)
    
    return formatted_message_history

In [19]:
formatted_history = format_message_history(sample_message_history)
print(formatted_history)

[01/10/2023, 10:15 AM] Isabel: Hey everyone! We should plan to watch the match on Tuesday! Who's in? ⚽️
[01/10/2023, 10:16 AM] Ana: I'm in! Can't wait to see the game! 🙌
[01/10/2023, 10:17 AM] Carmen: I have plans this weekend, so I can't make it. 😞
[01/10/2023, 10:18 AM] Julia: Count me in! Where are we watching it? 
[01/10/2023, 10:19 AM] Laura: I’ll join too! Let’s make it a fun day! 🎉
[01/10/2023, 10:20 AM] Sofia: I might be able to come, but I’ll confirm later. 
[01/10/2023, 10:21 AM] Isabel: Great! Let’s meet at my place then. 
[01/10/2023, 10:22 AM] Isabel: Also, there’s a friendly match next Friday. Anyone interested?
[01/10/2023, 10:23 AM] Ana: I can’t make it on Friday. I have a meeting. 😩
[01/10/2023, 10:24 AM] Carmen: I’m free! I’d love to join! 
[01/10/2023, 10:25 AM] Julia: I’ll be there too! 
[01/10/2023, 10:26 AM] Laura: I can’t do Wednesday, unfortunately. 
[01/10/2023, 10:27 AM] Sofia: I’ll join for the friendly match! 
[01/10/2023, 10:28 AM] Isabel: Awesome! So far w

In [27]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv
from agent.prompts import SYSTEM_PROMPT

load_dotenv()

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str = Field(description="The name of the event")
    event_date: str = Field(description="The date of the event. Not the date the message timestamp.")
    time: str = Field(description="The time of the event")
    participants: list[str] = Field(description="The participants of the event")
    number_of_participants: int = Field(description="The number of participants of the event")
    not_attending: list[str] = Field(description="The people who are not attending the event")
    didnt_confirm: list[str] = Field(description="The people who didn't confirm the event")
    location: str = Field(description="The location of the event")

class CalendarEvents(BaseModel):
    events: list[CalendarEvent]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": formatted_history},
    ],
    response_format=CalendarEvents,
)

response = completion.choices[0].message.parsed

In [317]:
pretty_print(completion.choices[0].message)

{'audio': None,
 'content': '{"events":[{"name":"Game from '
            'yesterday","event_date":"03/02/25","time":"N/A","participants":["Mario","Carmen","Maria","Sofia","Ana","Julia","Isabel","Paula","Victoria"],"number_of_participants":9,"not_attending":["Elena"],"didnt_confirm":[],"location":"Indoor '
            'court"},{"name":"Pizza '
            'night","event_date":"04/02/25","time":"20:00","participants":["Julia","Maria","Sofia","Ana","Laura","Carmen","Elena","Isabel","Paula"],"number_of_participants":9,"not_attending":["Victoria"],"didnt_confirm":[],"location":"Julia\'s '
            'place"},{"name":"Sauna '
            'outing","event_date":"13/02/25","time":"N/A","participants":["Sofia","Ana","Julia","Elena","Isabel"],"number_of_participants":5,"not_attending":["Maria","Victoria"],"didnt_confirm":["Paula"],"location":"N/A"}]}',
 'function_call': None,
 'parsed': {'events': [{'didnt_confirm': [],
                        'event_date': '03/02/25',
                        'l

In [318]:
response.events

[CalendarEvent(name='Game from yesterday', event_date='03/02/25', time='N/A', participants=['Mario', 'Carmen', 'Maria', 'Sofia', 'Ana', 'Julia', 'Isabel', 'Paula', 'Victoria'], number_of_participants=9, not_attending=['Elena'], didnt_confirm=[], location='Indoor court'),
 CalendarEvent(name='Pizza night', event_date='04/02/25', time='20:00', participants=['Julia', 'Maria', 'Sofia', 'Ana', 'Laura', 'Carmen', 'Elena', 'Isabel', 'Paula'], number_of_participants=9, not_attending=['Victoria'], didnt_confirm=[], location="Julia's place"),
 CalendarEvent(name='Sauna outing', event_date='13/02/25', time='N/A', participants=['Sofia', 'Ana', 'Julia', 'Elena', 'Isabel'], number_of_participants=5, not_attending=['Maria', 'Victoria'], didnt_confirm=['Paula'], location='N/A')]

In [319]:
for event in response.events:
    print(event.model_dump_json())

{"name":"Game from yesterday","event_date":"03/02/25","time":"N/A","participants":["Mario","Carmen","Maria","Sofia","Ana","Julia","Isabel","Paula","Victoria"],"number_of_participants":9,"not_attending":["Elena"],"didnt_confirm":[],"location":"Indoor court"}
{"name":"Pizza night","event_date":"04/02/25","time":"20:00","participants":["Julia","Maria","Sofia","Ana","Laura","Carmen","Elena","Isabel","Paula"],"number_of_participants":9,"not_attending":["Victoria"],"didnt_confirm":[],"location":"Julia's place"}
{"name":"Sauna outing","event_date":"13/02/25","time":"N/A","participants":["Sofia","Ana","Julia","Elena","Isabel"],"number_of_participants":5,"not_attending":["Maria","Victoria"],"didnt_confirm":["Paula"],"location":"N/A"}


In [320]:
import pandas as pd
import json

pd.DataFrame([json.loads(event.model_dump_json()) for event in response.events])


,name,event_date,time,participants,number_of_participants,not_attending,didnt_confirm,location
0,Game from yesterday,03/02/25,N/A,"[Mario, Carmen, Maria, Sofia, Ana, Julia, Isab...",9,[Elena],[],Indoor court
1,Pizza night,04/02/25,20:00,"[Julia, Maria, Sofia, Ana, Laura, Carmen, Elen...",9,[Victoria],[],Julia's place
2,Sauna outing,13/02/25,N/A,"[Sofia, Ana, Julia, Elena, Isabel]",5,"[Maria, Victoria]",[Paula],N/A


In [321]:
# process the messages to extract events
class CalendarEvent(BaseModel):
    name: str
    date: str
    time: str
    participants: str
    location: str
    number_of_participants: int

class CalendarEvents(BaseModel):
    events: list[CalendarEvent]

def process_messages(message_history):
    
    # Initialize the OpenAI client
    client = OpenAI()

    # Make the OpenAI API call to extract the events
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": sample_message_history},
        ],
        response_format=CalendarEvents,
    )

    # Parse the response
    response = completion.choices[0].message.parsed

    return pd.DataFrame([json.loads(event.model_dump_json()) for event in response.events])


In [322]:
events_df = process_messages(sample_message_history)

events_df

,name,date,time,participants,location,number_of_participants
0,Game from yesterday,03/02/25,N/A,Maria,Indoor court,8
1,Pizza night,04/02/25,20:00,Julia,Julia's place,6
2,Sauna,13/02/25,N/A,Sofia,N/A,4


In [323]:
from datetime import datetime

datetime.now().strftime("%A")

'Friday'